In [ ]:
#import of our defined environment
from environment import TradingEnv

In [ ]:
""" example of how to use the described environment
env = TradingEnv("AAPL", "1d", sliding_window=10, start_date="2020-01-01", end_date="2021-01-01")
obs, info = env.reset()
action = env.action_space.sample()  # es. 0=buy,1=hold,2=sell
obs, reward, done, truncated, info = env.step(action)
"""

In [ ]:
env = TradingEnv("AAPL", "1d", sliding_window=10, start_date="2020-01-01", end_date="2020-05-06")

state_size = env.observation_space.shape
action_size = env.action_space.n

print('State Space: ', state_size)
print('Action Space: ', action_size)

In [ ]:
# pip install torch

In [ ]:
import torch
import numpy as np
import random

# set the seeds for reproducibility of results
seed = 34
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
env.np_random = np.random.Generator(np.random.PCG64(seed))

env.reset(seed=seed)
env.action_space.seed(seed)
env.observation_space.seed(seed)

In [ ]:
def random_pi(state):
    # selects an action uniformly at random
    # from the environment's action space.
    return env.action_space.sample()

In [ ]:
# set the backend device to MPS, if available
if torch.backends.mps.is_available():
    device = torch.device("gpu")
else:
    device = torch.device("cpu")
# print the used device
print(f"Using device: {device}")

In [ ]:
# define the structured dtype for an experience tuple
experience_type = np.dtype([
    ('state',      np.float32, state_size),   # current state
    ('action',     np.int8),                  # action taken
    ('reward',     np.float32),               # reward received
    ('next_state', np.float32, state_size),   # next state
    ('failure',    np.int8)                   # terminal flag (1 if done)
])

In [ ]:
# Set the replay memory size
memory_size = 100000; 

# Create the replay memory
replay_memory = {
    'size': memory_size,
    'buffer': np.empty(shape=(memory_size,), dtype=experience_type),
    'index': 0,
    'entries': 0
}

In [ ]:
def store_experience(experience):
    # store the experience in the buffer
    replay_memory['buffer'][replay_memory['index']] = experience

    # update the number of experiences in the buffer
    replay_memory['entries'] = min(replay_memory['entries'] + 1, replay_memory['size'])

    # update index, if the memory is full, start from the begging
    replay_memory['index'] += 1
    replay_memory['index'] = replay_memory['index'] % replay_memory['size']

In [ ]:
# Set the batch size for sampling experiences
batch_size = 32

# function to sample a batch of experiences from the replay memory
def sample_experiences():

    # select uniformly at random a batch of experiences from the memory
    idxs = np.random.choice(range(replay_memory['entries']), batch_size, replace=False)

    # return the batch of experiences
    experiences = replay_memory['buffer'][idxs]

    return experiences

In [ ]:
import torch

first_hidden_layer= 512
second_hidden_layer= 128

def create_network():

      # Define a deep neural network using Sequential:
      # Each layer feeds directly into the next one.
      dnn = torch.nn.Sequential( 
            # First fully connected layer maps state inputs to 512 hidden units
            torch.nn.Linear(state_size[0], first_hidden_layer),
            
            # ReLU activation introduces nonlinearity
            torch.nn.ReLU(),
            
            # Second fully connected layer (hidden layer with 128 units)
            torch.nn.Linear(first_hidden_layer, second_hidden_layer),
            
            # Another ReLU activation
            torch.nn.ReLU(),
            
            # Output layer: one unit per possible action
            # Produces Q-values
            torch.nn.Linear(second_hidden_layer, action_size)
      )
    
      # Return the constructed model
      return dnn

In [ ]:
online_q = create_network()
target_q = create_network()

In [ ]:
learning_rate = 0.007
optimizer = torch.optim.RMSprop(online_q.parameters(), lr=learning_rate)

In [ ]:
def update_target():
    # copy the parameters from the online model to the target model
    for target, online in zip(target_q.parameters(), online_q.parameters()):
        target.data.copy_(online.data)

In [ ]:
def dqn_pi(state):
    # convert the state into a tensor
    state = torch.as_tensor(state, dtype=torch.float32)

    # compute Q-values from the network
    q_values = online_q(state).detach().numpy().squeeze()

    # select greedy action
    action = int(np.argmax(q_values))

    # return the action
    return action

In [ ]:
def evaluate(pi, episodes=1):

     # collect total rewards per episode
    rewards = []

    # loop over episodes
    for episode in range(episodes):

        # reset the environment
        state, _ = env.reset()
        done = False
        total_reward = 0.0

        # run an episode
        while not done:
            action = pi(state)
            state, reward, terminal, truncated, _ = env.step(action)
            total_reward += reward
            done = terminal or truncated

        # store the total reward    
        rewards.append(total_reward)
            
    # return the average reward over the episodes        
    return np.mean(rewards)

In [ ]:
# define the discount factor
gamma = 0.99

def optimize():

    # sample a batch of experiences
    batch = sample_experiences()
    
    # prepare the experience as tensors
    states      = torch.from_numpy(batch['state'].copy()).float()    
    actions     = torch.from_numpy(batch['action'].copy()).long()   
    rewards     = torch.from_numpy(batch['reward'].copy()).float()    
    next_states = torch.from_numpy(batch['next_state'].copy()).float() 
    failures    = torch.from_numpy(batch['failure'].copy()).float()

    # get the values of the Q-function at next state from the "target" network 
    # remember to detach, we need to treat these values as constants 
    q_target_next = target_q(next_states).detach()
    
    # get the max value 
    max_q_target_next = q_target_next.max(1)[0]

    # one important step, often overlooked, is to ensure 
    # that failure states are grounded to zero
    max_q_target_next *= (1 - failures.float())

    # calculate the target 
    target = rewards + gamma * max_q_target_next

    # finally, we get the current estimate of Q(s,a)
    # here we query the current "online" network
    q_online_current = torch.gather(online_q(states), 1, actions.unsqueeze(1)).squeeze(1)

    # create the errors
    td_error = target - q_online_current

    # calculate the loss
    loss = td_error.pow(2).mean()

    # backward pass: compute the gradients
    optimizer.zero_grad()
    loss.backward()

    # update model parameters
    optimizer.step()

In [ ]:
# define decay parameters (max, min, steps)
epsilon_max = 1.0
epsilon_min = 0.01
epsilon_decay_steps = 10000

# generate epsilons
epsilons = np.logspace(start=0, stop=-2, num=epsilon_decay_steps, base=10)
   
# normalize epsilons 
epsilons = (epsilons - epsilon_min) / (epsilon_max - epsilon_min)
    
# scale  epsilons to the desired range
epsilons = (epsilon_max - epsilon_min) * epsilons + epsilon_min

In [ ]:
import random

def epsilon_greedy(state, step):
    # get the epsilon value    
    epsilon = epsilons[step] if step < epsilon_decay_steps else epsilon_min

    # Exploration
    if random.random() < epsilon:
        action = random_pi(state)

    # Exploitation
    else:
        action = dqn_pi(state)
    
    return action

In [ ]:
def dqn(memory_start_size, target_update_steps, max_episodes):
    
    # create a score tracker for statistic purposes
    scores = []
    
    # counter for the number of steps 
    step = 0

    # update the target model with the online one
    update_target()
                   
    # train until the maximum number of episodes
    for episode in range(max_episodes):
        
        # reset the environment before starting the episode
        state, _ = env.reset()
        done = False

        # interact with the environment until the episode is done
        while not done:
                    
            # select the action using the exploration policy
            action = epsilon_greedy(state, step)

            # perform the selected action
            next_state, reward, terminal, truncated, _ = env.step(action)
            done = terminal or truncated
            failure = terminal and not truncated

            # store the experience into the replay buffer
            experience = (state, action, reward, next_state, failure)
            store_experience(experience)
    
            # optimize the online model after the replay buffer is large enough
            if replay_memory['entries'] > memory_start_size:
                optimize()
                 
                # sometimes, synchronize the target model with the online model
                if step % target_update_steps == 0:
                    update_target()
                
            # update current state to next state
            state = next_state

            # update the step counter
            step += 1

        # After each episode, evaluate the policy
        score = evaluate(dqn_pi, episodes=10)

        # store the score in the tracker
        scores.append(score)

        # print some informative logging 
        message = 'Episode {:03}, score {:05.1f}'
        message = message.format(episode+1, score)
        print(message, end='\r', flush=True)
        
    return scores

In [ ]:
# set the hyperparameters
memory_start_size = 1000
max_episodes = 25 #TODO! 100
target_update_steps = 10

# run the DQN algorithm
dqn(memory_start_size, target_update_steps, max_episodes)

In [ ]:
import joblib

def experiment(max_episodes):

    global online_q, target_q, optimizer, replay_memory, epsilons

    # List of random seeds to test algorithm stability
    seeds = (12, 34, 56, 78, 90)

    # Container to collect all experiment results
    results = []

    # Run an independent training experiment per seed
    for seed in seeds:

        print("Experiment seed: ", seed)

         # Set all relevant random seeds for reproducibility
        torch.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)

        # reset the environment
        env.reset(seed=seed)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)

        # create online and target models
        online_q = create_network()
        target_q = create_network()
        optimizer = torch.optim.RMSprop(online_q.parameters(), lr=learning_rate)

        # create the replay memory
        replay_memory = {
            'size': memory_size,
            'buffer': np.empty(shape=(memory_size,), dtype=experience_type),
            'index': 0,
            'entries': 0
        }

        # create the epsilon values
        epsilons = np.logspace(start=0, stop=-2, num=epsilon_decay_steps, base=10)
        epsilons = (epsilons - epsilon_min) / (epsilon_max - epsilon_min)
        epsilons = (epsilon_max - epsilon_min) * epsilons + epsilon_min

        # train the network    
        scores = dqn(memory_start_size, target_update_steps, max_episodes)
        
        # smooth the result using a sliding window
        sliding_windows = 25
        scores = np.convolve(scores, np.ones(sliding_windows)/sliding_windows, mode='valid')
                
        # collect the results
        results.append(scores)

        print("")

    # calculate max, min and average scores among experiments
    max_score = np.max(results, axis=0).T
    min_score = np.min(results, axis=0).T
    mean_score = np.mean(results, axis=0).T

    # prepare the results
    experiment_results = {
        'max_score': max_score,
        'min_score': min_score,
        'mean_score': mean_score
    }

    # save permanently
    joblib.dump(experiment_results, '../dqn_results.joblib');
    
    return experiment_results

In [ ]:
# Define the experiment setup and hyperparameters
gamma = 0.99;               # discount factor
learning_rate = 0.001;      # step size for the optimizer
batch_size = 512;          # number of experiences per batch
epochs = 8;                # optimization steps per batch
epsilon = 0.5               # esploration vs exploitation parameter
first_hidden_layer = 256;   # size of the first hidden layer
second_hidden_layer = 128;  # size of the second hidden layer

# Run the experiment
dqn_results = experiment(max_episodes=1) #TODO! 1500

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
plt.title('DQN cumulative reward')
plt.ylabel('Aggregated reward [$]')
plt.xlabel('Episodes')

episodes = range(len(dqn_results['max_score']))

plt.plot(dqn_results['max_score'], 'b', linewidth=1, label="DQN")
plt.plot(dqn_results['min_score'], 'b', linewidth=1)
plt.plot(dqn_results['mean_score'], 'b', linewidth=2)
plt.fill_between(episodes, dqn_results['min_score'], dqn_results['max_score'], facecolor='b', alpha=0.3)

plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

max_s = np.asarray(dqn_results['max_score'])
min_s = np.asarray(dqn_results['min_score'])
mean_s = np.asarray(dqn_results['mean_score'])

episodes = np.arange(len(mean_s))

plt.figure(figsize=(12, 6))
plt.plot(episodes, mean_s, label='DQN mean', linewidth=2)
plt.fill_between(episodes, min_s, max_s, alpha=0.3, label='min–max range')

plt.title('DQN cumulative reward')
plt.xlabel('Episodes')
plt.ylabel('Aggregated reward [$]')
plt.legend()
plt.grid(True)
plt.show()
